In [1]:
!pip -q install --upgrade torch transformers datasets evaluate accelerate

In [1]:

import pandas as pd, torch, datasets
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

DATA_PATH = "data/processed/train.parquet"  
LABELS = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]
df = pd.read_parquet(DATA_PATH)
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42, stratify=df["toxic"])
print(train_df.shape, val_df.shape)

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


c:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\__init__.py:16: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.0)
  from scipy.sparse import issparse


(143613, 8) (15958, 8)


In [2]:

def df_to_dataset(df):
    encodings = tokenizer(
        df["comment_text"].tolist(),
        truncation=True,
        padding=True,
        max_length=128,
    )
    encodings["labels"] = df[LABELS].astype("float32").values.tolist()
    return datasets.Dataset.from_dict(encodings)

train_ds = df_to_dataset(train_df)
val_ds   = df_to_dataset(val_df)
train_ds, val_ds


(Dataset({
     features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
     num_rows: 143613
 }),
 Dataset({
     features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
     num_rows: 15958
 }))

In [3]:
import torch, platform
print("CUDA available:", torch.cuda.is_available())
print("Detected GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

CUDA available: True
Detected GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [18]:

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import f1_score

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(LABELS),
    problem_type="multi_label_classification"
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.sigmoid(torch.tensor(logits))
    preds = (probs > 0.5).int().numpy()
    macro_f1 = f1_score(labels, preds, average="macro")
    return {"macro_f1": macro_f1}

training_args = TrainingArguments(
    output_dir="./bert_runs",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    logging_steps=2000,      
    save_steps=2000,         
    do_eval=True,           
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Administrator\AppData\Local\Temp\ipykernel_5964\4073449612.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
2000,0.068600
4000,0.051500
6000,0.046600
8000,0.043500
10000,0.042900
12000,0.042800
14000,0.041300
16000,0.041200
18000,0.039500
20000,0.031800


TrainOutput(global_step=53856, training_loss=0.0337522425824379, metrics={'train_runtime': 8152.9077, 'train_samples_per_second': 52.845, 'train_steps_per_second': 6.606, 'total_flos': 2.834064379532851e+16, 'train_loss': 0.0337522425824379, 'epoch': 3.0})

In [19]:

metrics = trainer.evaluate()
print(metrics)


{'eval_loss': 0.044837553054094315, 'eval_macro_f1': 0.6749728084779072, 'eval_runtime': 59.7263, 'eval_samples_per_second': 267.186, 'eval_steps_per_second': 33.402, 'epoch': 3.0}


In [20]:
def predict(text, thresh=0.5):

    device = next(model.parameters()).device         

    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128,
    ).to(device)

    with torch.no_grad():
        logits = model(**inputs).logits
    probs = torch.sigmoid(logits)[0].cpu().numpy()    

    prob_dict = {lab: float(p) for lab, p in zip(LABELS, probs)}
    prob_dict["prediction"] = [lab for lab, p in zip(LABELS, probs) if p > thresh]
    return prob_dict


In [28]:
predict("You are the worst!")

{'toxic': 0.9844720959663391,
 'severe_toxic': 0.00387908355332911,
 'obscene': 0.04676353186368942,
 'threat': 0.0010089100105687976,
 'insult': 0.9472251534461975,
 'identity_hate': 0.007398299407213926,
 'prediction': ['toxic', 'insult']}

In [ ]:
# LOAD CHECKPOINT AND EVALUATE FROM HERE

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# --- Settings ---
LABELS = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]
CHECKPOINT = "./bert_runs/checkpoint-53856" 
MODEL_NAME = "bert-base-uncased"
TEST_CSV = "test_samples.csv"    
OUT_CSV = "unseen_test_with_preds.csv"

# --- Load Model & Tokenizer ---
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# --- Prediction Helper ---
def predict_batch(texts, thresh=0.5):
    enc = tokenizer(
        texts, 
        return_tensors="pt", 
        truncation=True, 
        padding=True, 
        max_length=128
    ).to(device)

    with torch.no_grad():
        logits = model(**enc).logits
        probs = torch.sigmoid(logits).cpu().numpy()
    preds = (probs > thresh).astype(int)
    return probs, preds

# --- Read Data ---
df = pd.read_csv(TEST_CSV)

# --- Predict in batches ---
BATCH_SIZE = 32
all_probs = []
all_preds = []

for i in range(0, len(df), BATCH_SIZE):
    batch_texts = df["comment_text"].iloc[i:i+BATCH_SIZE].tolist()
    probs, preds = predict_batch(batch_texts)
    all_probs.append(probs)
    all_preds.append(preds)

# Stack and add to df
probs = np.vstack(all_probs)
preds = np.vstack(all_preds)

for i, label in enumerate(LABELS):
    df[label] = probs[:, i]
df["prediction"] = [
    [LABELS[j] for j, p in enumerate(row) if p == 1] 
    for row in preds
]

# --- Save ---
df.to_csv(OUT_CSV, index=False)
print("Done! Saved predictions to:", OUT_CSV)


Done! Saved predictions to: unseen_test_with_preds.csv


In [ ]:
import pandas as pd

samples = pd.read_csv("test_samples.csv")
labels  = pd.read_csv("test_labels.csv")

merged = samples.merge(labels, on="id", how="left")

print(merged.head())
merged.to_csv("test_samples_with_labels.csv", index=False)


                 id                                       comment_text  toxic  \
0  d4ede52e6fc315a2  == Sky city == \n\n Hello, \n\n I noticed that...      0   
1  345bb5c81a8a6797  " \n\n ""Specifically, Allied forces suffered ...      0   
2  73ef403fa5a095e6  " \n You know what, I think you're just insecu...     -1   
3  d5290e73cf136173  " \n\n == LOL == \n\n LOLLOLOLOLOOLOLLOLOLOLLO...     -1   
4  d4911d1fd9989582  he is not a bad guy he is a sexy guy and his b...     -1   

   severe_toxic  obscene  threat  insult  identity_hate  
0             0        0       0       0              0  
1             0        0       0       0              0  
2            -1       -1      -1      -1             -1  
3            -1       -1      -1      -1             -1  
4            -1       -1      -1      -1             -1  


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    confusion_matrix,
    precision_recall_fscore_support,
    roc_auc_score,
    accuracy_score,
    classification_report,
)

LABELS = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]

preds = pd.read_csv("unseen_test_with_preds.csv")
truth = pd.read_csv("test_samples_with_labels.csv")

df = preds.merge(truth[["id"] + LABELS], on="id", suffixes=("_pred", "_true"))

for label in LABELS:
    df[label + "_pred_bin"] = (df[label + "_pred"] > 0.5).astype(int)

mask_matrix = []
y_true_matrix = []
y_pred_matrix = []
y_prob_matrix = []

for _, row in df.iterrows():
    y_true_row = []
    y_pred_row = []
    y_prob_row = []
    mask_row = []
    for label in LABELS:
        gt = row[label + "_true"]
        if gt != -1:
            mask_row.append(True)
            y_true_row.append(gt)
            y_pred_row.append(row[label + "_pred_bin"])
            y_prob_row.append(row[label + "_pred"])
        else:
            mask_row.append(False)
            y_true_row.append(0)  
            y_pred_row.append(0)
            y_prob_row.append(0)
    y_true_matrix.append(y_true_row)
    y_pred_matrix.append(y_pred_row)
    y_prob_matrix.append(y_prob_row)
    mask_matrix.append(mask_row)

y_true_matrix = np.array(y_true_matrix)
y_pred_matrix = np.array(y_pred_matrix)
y_prob_matrix = np.array(y_prob_matrix)
mask_matrix = np.array(mask_matrix, dtype=bool)

def get_valid(label_idx):
    valid = mask_matrix[:, label_idx]
    return y_true_matrix[valid, label_idx], y_pred_matrix[valid, label_idx], y_prob_matrix[valid, label_idx]

print("="*40)
print("Per-label confusion matrices:")
for i, label in enumerate(LABELS):
    y_true_lbl, y_pred_lbl, _ = get_valid(i)
    cm = confusion_matrix(y_true_lbl, y_pred_lbl)
    print(f"\n{label}\n", pd.DataFrame(cm, index=["True 0", "True 1"], columns=["Pred 0", "Pred 1"]))

print("\nPer-label Precision, Recall, F1, Accuracy, ROC-AUC:")
stats = []
for i, label in enumerate(LABELS):
    y_true_lbl, y_pred_lbl, y_prob_lbl = get_valid(i)
    prec, rec, f1, _ = precision_recall_fscore_support(
        y_true_lbl, y_pred_lbl, average='binary', zero_division=0
    )
    acc = accuracy_score(y_true_lbl, y_pred_lbl)
    try:
        auc = roc_auc_score(y_true_lbl, y_prob_lbl)
    except Exception:
        auc = np.nan
    stats.append([label, acc, prec, rec, f1, auc])
df_stats = pd.DataFrame(stats, columns=["Label", "Accuracy", "Precision", "Recall", "F1", "ROC-AUC"]).set_index("Label")
print(df_stats.round(3))

print("\nMacro/Micro Precision, Recall, F1, ROC-AUC:")
flat_true, flat_pred, flat_prob = [], [], []
for i in range(len(LABELS)):
    y_true_lbl, y_pred_lbl, y_prob_lbl = get_valid(i)
    flat_true += list(y_true_lbl)
    flat_pred += list(y_pred_lbl)
    flat_prob += list(y_prob_lbl)

prec_macro, rec_macro, f1_macro, _ = precision_recall_fscore_support(flat_true, flat_pred, average="macro", zero_division=0)
prec_micro, rec_micro, f1_micro, _ = precision_recall_fscore_support(flat_true, flat_pred, average="micro", zero_division=0)
try:
    roc_auc_macro = roc_auc_score(flat_true, flat_prob, average="macro")
except Exception:
    roc_auc_macro = np.nan

print(f"Macro: Precision={prec_macro:.3f}, Recall={rec_macro:.3f}, F1={f1_macro:.3f}, ROC-AUC={roc_auc_macro:.3f}")
print(f"Micro: Precision={prec_micro:.3f}, Recall={rec_micro:.3f}, F1={f1_micro:.3f}")

masked_y_true = np.where(mask_matrix, y_true_matrix, np.nan)
masked_y_pred = np.where(mask_matrix, y_pred_matrix, np.nan)

report_dict = {}
for i, label in enumerate(LABELS):
    y_true_lbl, y_pred_lbl, _ = get_valid(i)
    report = classification_report(y_true_lbl, y_pred_lbl, output_dict=True, zero_division=0)
    report_dict[label] = report['1']

df_report = pd.DataFrame(report_dict).T[['precision', 'recall', 'f1-score', 'support']]
print("\nScikit-learn per-label classification report:")
print(df_report.round(3))

sample_acc = []
for row_true, row_pred, row_mask in zip(y_true_matrix, y_pred_matrix, mask_matrix):
    n_valid = row_mask.sum()
    if n_valid == 0:
        continue
    sample_acc.append((row_true[row_mask] == row_pred[row_mask]).mean())
print(f"\nMean samplewise accuracy (fraction of correct labels per row): {np.mean(sample_acc):.3f}")

all_valid_mask = mask_matrix.all(axis=1)
if all_valid_mask.sum():
    subset_acc = np.mean([np.array_equal(y_true_matrix[i], y_pred_matrix[i]) for i in range(len(y_true_matrix)) if all_valid_mask[i]])
    print(f"Subset accuracy (exact match on all labels, only full rows): {subset_acc:.3f}")

print("="*40)


Per-label confusion matrices:

toxic
         Pred 0  Pred 1
True 0     338      25
True 1       1      36

severe_toxic
         Pred 0  Pred 1
True 0     393       3
True 1       1       3

obscene
         Pred 0  Pred 1
True 0     373       4
True 1       6      17

threat
         Pred 0  Pred 1
True 0     398       0
True 1       1       1

insult
         Pred 0  Pred 1
True 0     371       7
True 1       6      16

identity_hate
         Pred 0  Pred 1
True 0     395       1
True 1       2       2

Per-label Precision, Recall, F1, Accuracy, ROC-AUC:
               Accuracy  Precision  Recall     F1  ROC-AUC
Label                                                     
toxic             0.935      0.590   0.973  0.735    0.984
severe_toxic      0.990      0.500   0.750  0.600    0.997
obscene           0.975      0.810   0.739  0.773    0.983
threat            0.998      1.000   0.500  0.667    1.000
insult            0.968      0.696   0.727  0.711    0.985
identity_hate     0.992